In [ ]:
import DeepFMKit.core as dfm
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import multiprocessing
import os
import pickle

# Import the new worker function from your workers.py file
from DeepFMKit.workers import calculate_stat_sys_tradeoff

def run_stat_sys_tradeoff_analysis(
    T_acq_range=np.logspace(-2, 2, 20),
    m_true=15.5,
    amp_asd=1e-5,    # V/sqrt(Hz)
    freq_asd=1000.0, # Hz/sqrt(Hz) @ 1 Hz
    n_trials=10,
    n_cores=None
):
    """
    Orchestrates the full analysis of statistical vs. systematic errors.
    """
    if n_cores is None:
        n_cores = os.cpu_count()

    print("=" * 60)
    print("Running Statistical vs. Systematic Trade-off Analysis")
    print(f"Acquisition Times: {len(T_acq_range)} points from {T_acq_range[0]:.2e} to {T_acq_range[-1]:.2e} s")
    print(f"Number of trials for each point: {n_trials}")
    print("=" * 60)
    
    jobs = []
    # Create jobs for the systematic bias runs (one set of trials for each T_acq)
    for T_acq in T_acq_range:
        for i in range(n_trials):
            jobs.append({
                'sim_type': 'sys_hybrid',
                'T_acq': T_acq,
                'm_true': m_true,
                'freq_asd': freq_asd,
                'trial_num': i
            })
    
    # Create jobs for the statistical uncertainty runs
    for T_acq in T_acq_range:
        for i in range(n_trials):
             jobs.append({
                'sim_type': 'stat',
                'T_acq': T_acq,
                'm_true': m_true,
                'amp_asd': amp_asd,
                'trial_num': i
            })

    # Run all jobs in parallel
    results_list = []
    if __name__ == "__main__":
        with multiprocessing.Pool(processes=n_cores) as pool:
            results_iterator = pool.imap(calculate_stat_sys_tradeoff, jobs)
            for result in tqdm(results_iterator, total=len(jobs), desc="Running Trials"):
                results_list.append(result)

    # --- Process and Structure the Results ---
    sys_results = {T: [] for T in T_acq_range}
    stat_results = {T: [] for T in T_acq_range}
    
    # Re-associate results with their parameters
    job_idx = 0
    for T_acq in T_acq_range:
        for _ in range(n_trials):
            sys_results[T_acq].append(results_list[job_idx])
            job_idx += 1
    for T_acq in T_acq_range:
        for _ in range(n_trials):
            stat_results[T_acq].append(results_list[job_idx])
            job_idx += 1

    # Calculate final statistics
    sys_bias = np.array([np.mean(sys_results[T]) for T in T_acq_range])
    stat_unc = np.array([np.std(stat_results[T]) for T in T_acq_range])
    
    final_data = {
        "T_acq_range": T_acq_range,
        "systematic_bias_vs_T": sys_bias,
        "statistical_unc_vs_T": stat_unc,
        "m_true": m_true
    }
    
    return final_data

In [ ]:
# --- Main execution block ---
if __name__ == "__main__":
    results_filename = 'tradeoff_analysis_results.pkl'
    
    # --- Run Simulation ---
    tradeoff_results = run_stat_sys_tradeoff_analysis(
        n_trials=1, # Use a lower number for faster testing
        T_acq_range=np.logspace(0, 2, 2)
    )
    
    # --- Save and Plot ---
    with open(results_filename, 'wb') as f:
        pickle.dump(tradeoff_results, f)

In [ ]:
def plot_stat_sys_tradeoff(results):
    """
    Plots the final "V-curve" showing the error tradeoff.
    """
    T_acq = results['T_acq_range']
    sys_bias = np.abs(results['systematic_bias_vs_T'])
    stat_unc = results['statistical_unc_vs_T']
    
    fig, ax = plt.subplots(figsize=(10, 7))

    ax.loglog(T_acq, stat_unc, 'o-', label=r'Statistical Uncertainty ($\delta m_{\rm stat}$)')
    ax.loglog(T_acq, sys_bias, 's-', label=r'Systematic Bias ($|\Delta m_{\rm sys}|$ from Drift)')
    
    # Add theoretical scaling lines for comparison
    # T^-0.5 for statistical noise
    stat_theory = stat_unc[0] * (T_acq[0] / T_acq)**0.5
    ax.loglog(T_acq, stat_theory, 'k--', alpha=0.7, label=r'$\propto T_{\rm acq}^{-1/2}$')
    
    # T^+0.5 for random walk induced bias (sqrt of integrated variance)
    sys_theory = sys_bias[2] * (T_acq / T_acq[2])**0.5
    ax.loglog(T_acq, sys_theory, 'k:', alpha=0.7, label=r'$\propto T_{\rm acq}^{+1/2}$')
    
    # Total error
    total_error = np.sqrt(stat_unc**2 + sys_bias**2)
    ax.loglog(T_acq, total_error, 'd-', color='black', lw=2, label='Total Error (in quadrature)')
    
    # Find and mark the optimal point
    opt_idx = np.argmin(total_error)
    opt_T = T_acq[opt_idx]
    opt_err = total_error[opt_idx]
    ax.plot(opt_T, opt_err, 'r*', markersize=15, label=f'Optimal Point\n({opt_T:.1f} s, {opt_err:.1e} rad)')
    
    ax.set_xlabel(r'Acquisition Time, $T_{\rm acq}$ (s)', fontsize=14)
    ax.set_ylabel(r'Uncertainty / Bias on $m$ (rad)', fontsize=14)
    ax.set_title('DFMI Performance: Statistical vs. Systematic Error Trade-off', fontsize=16)
    ax.grid(True, which='both', linestyle='--')
    ax.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
with open(results_filename, 'rb') as f:
    loaded_results = pickle.load(f)
plot_stat_sys_tradeoff(loaded_results)